# Assignment 5

In [1]:
#imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

from sklearn import datasets
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler

import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasRegressor

## Part 1: Implement a Perceptron

Given the diabetes dataset you used during Assignment 2, implement an
MLP Regressor.

In [2]:
# Assignment 2 used the sklearn diabetes dataset:
diabetes = datasets.load_diabetes()
#load into padas df:
features = pd.DataFrame(diabetes.data, columns = diabetes.feature_names)
target = diabetes.target
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   age     442 non-null    float64
 1   sex     442 non-null    float64
 2   bmi     442 non-null    float64
 3   bp      442 non-null    float64
 4   s1      442 non-null    float64
 5   s2      442 non-null    float64
 6   s3      442 non-null    float64
 7   s4      442 non-null    float64
 8   s5      442 non-null    float64
 9   s6      442 non-null    float64
dtypes: float64(10)
memory usage: 34.7 KB


In [3]:
#split the data:
Xtrain, Xtest, ytrain, ytest = train_test_split(features, target, random_state=8, test_size = 0.2)
Xtrain.head()
#data does not need to be scaled 

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
416,-0.027310,-0.044642,0.080019,0.098751,-0.002945,0.018101,-0.017629,0.003312,-0.029526,0.036201
141,-0.027310,-0.044642,0.047685,-0.046985,0.034206,0.057245,-0.080217,0.130252,0.045067,0.131470
127,0.034443,0.050680,-0.001895,-0.012556,0.038334,0.013717,0.078093,-0.039493,0.004548,-0.096346
120,-0.049105,-0.044642,0.004572,0.011544,-0.037344,-0.018537,-0.017629,-0.002592,-0.039809,-0.021788
313,0.059871,0.050680,0.053074,0.052858,0.032830,0.019667,-0.010266,0.034309,0.055203,-0.001078


In [4]:
#Train MLP Regressor:
diabetesReg = MLPRegressor(hidden_layer_sizes=(300,200,20), random_state=8, max_iter=5000).fit(Xtrain,ytrain)

pred1 = diabetesReg.predict(Xtest)
score1 = diabetesReg.score(Xtest,ytest)

score1

0.20972290728939047

## Part 2: Implement a Keras Classifier

Given the prepared Titanic dataset from Assignment 3, implement a Keras
sequential classifier with relu activation functions.

In [5]:
# Assignment 3 used the provided titanic dataset
titanic = pd.read_csv('train.csv')
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [6]:
# Clean the data as we did in assignment 3:
titanic = titanic.drop('Cabin', axis = 1)

def impute_age(cols):
    Age=cols[0]
    Pclass=cols[1]
    if pd.isnull(Age):
        if Pclass==1:
            return 37
        elif Pclass==2:
            return 29
        else:
            return 24
    else:
        return Age
titanic['Age'] = titanic[['Age','Pclass']].apply(impute_age, axis=1)

titanic.dropna(axis=0, inplace=True)
titanic.isna().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64

In [7]:
# dataset has categorical values:
titanic = pd.get_dummies(titanic, columns = ['Sex', 'Embarked'])
titanic = titanic.drop(['Name', 'Ticket', 'PassengerId'], axis = 1)

titanic.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,3,22.0,1,0,7.2500,0,1,0,0,1
1,1,1,38.0,1,0,71.2833,1,0,1,0,0
2,1,3,26.0,0,0,7.9250,1,0,0,0,1
3,1,1,35.0,1,0,53.1000,1,0,0,0,1
4,0,3,35.0,0,0,8.0500,0,1,0,0,1


In [8]:
# train test split:
feature2 = titanic.drop('Survived', axis = 1)
target2 = titanic['Survived'].copy()
Xtrain2, Xtest2, ytrain2, ytest2 = train_test_split(feature2, target2, random_state = 8)
Xtrain2.shape

(666, 10)

In [9]:
#build sequential classifier model:
titanicKSC = Sequential()
titanicKSC.add(Dense(300, activation='relu', input_dim=10))
titanicKSC.add(Dense(100, activation='relu'))
titanicKSC.add(Dense(2, activation='softmax'))

#compile
titanicKSC.compile(loss='sparse_categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [10]:
# fit model
titanicKSC.fit(Xtrain2,ytrain2, epochs = 30)

Epoch 1/30
21/21 [==============================] - 1s 2ms/step - loss: 0.8789 - accuracy: 0.6802
Epoch 2/30
21/21 [==============================] - 0s 3ms/step - loss: 0.7126 - accuracy: 0.6862
Epoch 3/30
21/21 [==============================] - 0s 3ms/step - loss: 0.6526 - accuracy: 0.7087
Epoch 4/30
21/21 [==============================] - 0s 3ms/step - loss: 0.5460 - accuracy: 0.7553
Epoch 5/30
21/21 [==============================] - 0s 3ms/step - loss: 0.4942 - accuracy: 0.7718
Epoch 6/30
21/21 [==============================] - 0s 3ms/step - loss: 0.5161 - accuracy: 0.7628
Epoch 7/30
21/21 [==============================] - 0s 3ms/step - loss: 0.5521 - accuracy: 0.7598
Epoch 8/30
21/21 [==============================] - 0s 3ms/step - loss: 0.5144 - accuracy: 0.7628
Epoch 9/30
21/21 [==============================] - 0s 2ms/step - loss: 0.4456 - accuracy: 0.7973
Epoch 10/30
21/21 [==============================] - 0s 3ms/step - loss: 0.5274 - accuracy: 0.7778
Epoch 11/30
21/21 [

In [11]:
#evaluate
pred2 = titanicKSC.predict(Xtest2)
score2 = titanicKSC.evaluate(Xtest2,ytest2)

score2

7/7 [==============================] - 0s 3ms/step - loss: 0.4790 - accuracy: 0.8072


[0.47896647453308105, 0.8071748614311218]

## Part 3: Implement a Keras Regressor

Given the prepared bike-share dataset from Assignment 4, implement a
Keras sequential regressor with relu activation functions.

In [12]:
# Assignment 4 used the provided bike stateset
bike = pd.read_csv('bike_share_hour.csv')

In [13]:
# clean dataset as we did in assignment 4
#categorical variables:
bike['season'] = bike['season'].astype('category')            
bike['yr'] = bike['yr'].astype('category')            
bike['holiday'] = bike['holiday'].astype('category')
bike['workingday'] = bike['workingday'].astype('category')            
bike['weathersit'] = bike['weathersit'].astype('category')            
bike['mnth'] = bike['mnth'].astype('category')            
bike['hr'] = bike['hr'].astype('category')            
bike['weekday'] = bike['weekday'].astype('category')            

# scale numerical features
num_features = ['temp', 'atemp', 'hum', 'windspeed']
scaler = StandardScaler()
scaler.fit(bike[num_features])
bike[num_features] = scaler.transform(bike[num_features])

# drop columns
bike = bike.drop(['casual', 'registered', 'dteday', 'instant'], axis = 1)
bike.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17379 entries, 0 to 17378
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   season      17379 non-null  category
 1   yr          17379 non-null  category
 2   mnth        17379 non-null  category
 3   hr          17379 non-null  category
 4   holiday     17379 non-null  category
 5   weekday     17379 non-null  category
 6   workingday  17379 non-null  category
 7   weathersit  17379 non-null  category
 8   temp        17379 non-null  float64 
 9   atemp       17379 non-null  float64 
 10  hum         17379 non-null  float64 
 11  windspeed   17379 non-null  float64 
 12  cnt         17379 non-null  int64   
dtypes: category(8), float64(4), int64(1)
memory usage: 817.0 KB


In [14]:
# train-test split
feature3 = bike.drop('cnt', axis = 1)
target3 = bike['cnt'].copy()
Xtrain3, Xtest3, ytrain3, ytest3 = train_test_split(feature3, target3, test_size = 0.33, random_state = 8)
Xtrain3.shape

(11643, 12)

In [15]:
# build sequential regressor model
def buildModel():
    bikeKSR = Sequential()
    bikeKSR.add(Dense(300, input_dim=12, activation='relu'))
    bikeKSR.add(Dense(100, activation='relu'))
    bikeKSR.add(Dense(30, activation='relu'))
    bikeKSR.add(Dense(1)) 
    bikeKSR.compile(loss='mean_squared_error', optimizer='adam')
    return bikeKSR

bikeKSR = KerasRegressor(build_fn=buildModel)

In [16]:
#fit
bikeKSR.fit(Xtrain3, ytrain3, epochs=30)

Epoch 1/30


C:\Users\mmall\AppData\Local\Programs\Python\Python310\lib\site-packages\scikeras\wrappers.py:301: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  warnings.warn(


364/364 [==============================] - 2s 3ms/step - loss: 24715.9414
Epoch 2/30
364/364 [==============================] - 1s 3ms/step - loss: 16431.7891
Epoch 3/30
364/364 [==============================] - 1s 3ms/step - loss: 14318.5527
Epoch 4/30
364/364 [==============================] - 1s 3ms/step - loss: 13412.9580
Epoch 5/30
364/364 [==============================] - 1s 3ms/step - loss: 12682.1221
Epoch 6/30
364/364 [==============================] - 1s 3ms/step - loss: 12120.2686
Epoch 7/30
364/364 [==============================] - 1s 3ms/step - loss: 11535.0215
Epoch 8/30
364/364 [==============================] - 1s 3ms/step - loss: 10942.5547
Epoch 9/30
364/364 [==============================] - 1s 3ms/step - loss: 10389.8545
Epoch 10/30
364/364 [==============================] - 1s 3ms/step - loss: 9667.9238
Epoch 11/30
364/364 [==============================] - 1s 3ms/step - loss: 9198.8145
Epoch 12/30
364/364 [==============================] - 1s 3ms/step - loss: 8

KerasRegressor(
	model=None
	build_fn=<function buildModel at 0x000001ABA2941090>
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=None
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=1
)

In [17]:
#evaluate
pred3 = bikeKSR.predict(Xtest3)
np.sqrt(mean_squared_error(ytest3,pred3))

180/180 [==============================] - 1s 2ms/step


57.60470080600907

## Part 4: Tune Your Keras Regressor

Tune your Keras regressor from Part 3 by implementing a grid search with
different optimizers.

In [18]:
# set up search
params = {'optimizer':['adam','sgd', 'nadam', 'adamax', 'adagrad', 'adadelta', 'ftrl', 'rmsprop']}
gridsearch = GridSearchCV(bikeKSR, param_grid=params, n_jobs=-1, cv=3, scoring='neg_mean_squared_error')

In [19]:
#fit
results = gridsearch.fit(Xtrain3,ytrain3, epochs=30)
results.best_estimator_

Epoch 1/30


C:\Users\mmall\AppData\Local\Programs\Python\Python310\lib\site-packages\scikeras\wrappers.py:301: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  warnings.warn(


364/364 [==============================] - 2s 3ms/step - loss: 24706.0527
Epoch 2/30
364/364 [==============================] - 1s 3ms/step - loss: 15989.8887
Epoch 3/30
364/364 [==============================] - 1s 3ms/step - loss: 14214.5059
Epoch 4/30
364/364 [==============================] - 1s 3ms/step - loss: 13174.3623
Epoch 5/30
364/364 [==============================] - 1s 3ms/step - loss: 12509.6650
Epoch 6/30
364/364 [==============================] - 1s 3ms/step - loss: 12199.6885
Epoch 7/30
364/364 [==============================] - 1s 3ms/step - loss: 11615.9600
Epoch 8/30
364/364 [==============================] - 1s 3ms/step - loss: 11281.6797
Epoch 9/30
364/364 [==============================] - 1s 3ms/step - loss: 10560.5381
Epoch 10/30
364/364 [==============================] - 1s 3ms/step - loss: 9915.0664
Epoch 11/30
364/364 [==============================] - 1s 3ms/step - loss: 9523.0889
Epoch 12/30
364/364 [==============================] - 1s 3ms/step - loss: 8

KerasRegressor(
	model=None
	build_fn=<function buildModel at 0x000001ABA2941090>
	warm_start=False
	random_state=None
	optimizer=adagrad
	loss=None
	metrics=None
	batch_size=None
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=1
)

In [20]:
#evaluate
pred4 = results.predict(Xtest3)
np.sqrt(mean_squared_error(ytest3,pred4))

180/180 [==============================] - 0s 2ms/step


59.85204368397318